# Superconducting Qubit Refrigerator: Paper Plots
This Jupyter Notebook produces all plots in the manuscript relative to the superconducting qubit refrigerator if the ```paper_plot_data``` was downloaded. Such data can alternatively be generated using ```0_train.ipynb``` and ```1_produce_pareto.ipynb```. For details on the system, see ```0_train.ipynb```.
#### Import modules

In [ ]:
import sys
import os
sys.path.append(os.path.join('..','src'))
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.offsetbox import AnchoredOffsetbox, TextArea, HPacker
from matplotlib.lines import Line2D
from pathlib import Path
import plotting

## Produce Fig. 3

In [ ]:
log_dir = "../paper_plot_data/qubit_refrigerator/2/2022_02_26-23_03_54_aend=0.6"

det_policy_sublocation = "saved_policies/det_policy.txt"
actions_to_plot_large = 110
actions_to_plot_small = 61
small_line_style = "scatter"
large_line_style = "scatter_plot"
small_linewidth=6.
large_linewidth=12.
actions_per_log = 1000
act_0 = 11000-1
act_1 = 140000-1
act_2 = 490000-1
small_action_ylim = [-0.05, 0.91]
large_action_ylim = None 
reward_ylabel= r""
reward_labels = [r"$\ev*{r_c}_i$",r"$\ev*{{P}_\text{cool}}_i/P_0$",
                                                         r"$-\ev*{\Sigma}_i/\Sigma_0$",""]
reward_extra_coeffs = [1.,1.,1.]
reward_label_location = [-0.0, 1.28]
reward_colors = ["black","limegreen","darkorange"]
cycle_custom_colors = ["black"]
reward_plot_extra_args = ([0,500000], [0.00149926,0.00149926])
reward_plot_extra_kwargs = dict(color='black',linewidth=0.8, dashes=(4/0.8,2/0.8))
extra_cycles = [lambda x,a=2,omega=0.0472428,dt=0.982: 0.25*(1. + np.tanh(a*np.cos(omega*x*dt))/np.tanh(a)),
               [0,110],"black"]
extra_cycles_linewidth = 0.8
rl_legend_lines=[Line2D([0], [0], color='black', linewidth=1), 
                    Line2D([0], [0], color='black', linewidth=1, dashes=(4/0.8,2/0.8))]
rl_legend_text=["RL","Trapezoidal"]
rl_legend_location=[0.738,0.882]
legend_column_spacing = 0.8
plot_file_name = "fig_3.pdf"

#get location of files
running_reward_file, running_loss_file, running_multi_obj_file, actions_file = \
                                plotting.log_sac_file_locations(log_dir, None,None,None,None)

#font size
matplotlib.rcParams.update({'font.size': 14, "text.usetex": True,
                            'text.latex.preamble' : r'\usepackage{amsmath,amssymb,physics}'})

#create the figure and subplots
fig = plt.figure(figsize=(6,5.9), constrained_layout=True)
fig.set_constrained_layout_pads(w_pad=0.01,wspace=0.)
gs = gridspec.GridSpec(4, 4, figure=fig, height_ratios=[1,0.5,0.7,1], width_ratios=[0.6,0.6,0.2,0.25])
reward_ax = fig.add_subplot(gs[0, :])
cvalue_ax = fig.add_subplot(gs[1, :],sharex=reward_ax)
prot_0_ax = fig.add_subplot(gs[2, 0])
prot_1_ax = fig.add_subplot(gs[2, 1],sharey=prot_0_ax)
prot_2_ax = fig.add_subplot(gs[2, 2:],sharey=prot_0_ax)
prot_final_ax = fig.add_subplot(gs[3, 0:3])
coupling_ax = fig.add_subplot(gs[3, 3], sharey=prot_final_ax)
plt.setp(reward_ax.get_xticklabels(), visible=False)   
plt.setp(prot_1_ax.get_yticklabels(), visible=False)
plt.setp(prot_2_ax.get_yticklabels(), visible=False)          
plt.setp(coupling_ax.get_yticklabels(), visible=False)          

#Figure of merit curve (panel a)
plotting.plot_running_reward_on_axis(running_reward_file, reward_ax, plot_to_file_line = None,
                                    multi_location=running_multi_obj_file,linewidth=None,
                                    lines_to_mark = [plotting.nearest_int(act_0/actions_per_log),
                                    plotting.nearest_int(act_1/actions_per_log),
                                    plotting.nearest_int(act_2/actions_per_log)], custom_mark_color="black",
                                    custom_mark_size=70, legend_labels=reward_labels,legend_location=reward_label_location,
                                    custom_colors=reward_colors,ylabel=reward_ylabel,legend_cols=3,
                                    extra_coeffs=reward_extra_coeffs,plot_extra_args=reward_plot_extra_args,
                                    plot_extra_kwargs=reward_plot_extra_kwargs,legend_column_spacing=legend_column_spacing,
                                    legend_fancybox=False, legend_framealpha=0.,xlabel="",yticks=[-1,0,1]) 

#Values of c during training (panel b)
plot_data = np.loadtxt(running_loss_file)
cvalue_ax.plot(plot_data[2:,0], plot_data[2:,5], color="black", linewidth=1.)
cvalue_ax.set_xlabel("step")
cvalue_ax.set_ylabel(r"$c$")
cvalue_ax.set_yticks([0.6,1])

#actions during training (panel c)
plotting.plot_actions_on_axis(actions_file, prot_0_ax, actions_to_plot=actions_to_plot_small,
                            is_tri=False, actions_ylim=small_action_ylim,plot_to_file_line=act_0,
                            custom_colors=cycle_custom_colors, linewidth = small_linewidth,
                            two_xticks=True, line_style=small_line_style)
plotting.plot_actions_on_axis(actions_file, prot_1_ax, actions_to_plot=actions_to_plot_small,
                            is_tri=False,plot_to_file_line=act_1,ylabel="",custom_colors=cycle_custom_colors,
                            linewidth = small_linewidth,two_xticks=True, line_style=small_line_style)
plotting.plot_actions_on_axis(actions_file, prot_2_ax, actions_to_plot=actions_to_plot_small,
                            is_tri=False,plot_to_file_line=act_2, ylabel="",custom_colors=cycle_custom_colors,
                            linewidth = small_linewidth, two_xticks=True, line_style=small_line_style)

#final deterministic cycle (panel d)
plotting.plot_actions_on_axis(os.path.join(log_dir, det_policy_sublocation), prot_final_ax,
                            actions_to_plot=actions_to_plot_large, is_tri=False, actions_ylim=large_action_ylim,
                            plot_to_file_line=None, custom_colors=cycle_custom_colors, line_style=large_line_style,
                            k_notation=False, x_count_from_zero=True, linewidth = large_linewidth, xlabel="$t[dt]$",
                            extra_cycles=extra_cycles, extra_cycles_linewidth=extra_cycles_linewidth)

#coupling strength functions (panel e)
s = lambda de, b, w: g/2 * 1/(1 + q**2*(de/w - w/de)**2)*de/(np.exp(b*de) - 1)
s_tot = lambda de, b, w: s(de,b,w)+s(-de,b,w)
s_hot = lambda de: s_tot(de,bh,wh)
s_cold = lambda de: s_tot(de,bc,wc)
de = lambda u: 2*e0*np.sqrt(d**2 + u**2);
g = 1;q = 4;e0 = 1;d = 0.12;wh = 1.03;wc = 0.24;bh = 10/3;bc = 2*bh;ec = 0.24;
u_vals = np.linspace(0,0.72,100)
hot_coupling = s_hot(de(u_vals))
cold_coupling = s_cold(de(u_vals))
coupling_ax.plot(hot_coupling, u_vals, color="red")
coupling_ax.plot(cold_coupling, u_vals, color="blue")
coupling_ax.set_xlim([0.,0.58])
xbox1 = TextArea(r"$\gamma^{(\text{C})}_{u(t)},$", textprops=dict(color="b", ha='center',va='bottom'))
xbox2 = TextArea(r"$\gamma^{(\text{H})}_{u(t)}$", textprops=dict(color="r", ha='center',va='bottom'))
xbox = HPacker(children=[xbox1, xbox2],align="bottom", pad=0, sep=5)
anchored_xbox = AnchoredOffsetbox(loc=3, child=xbox, pad=0., frameon=False, bbox_to_anchor=(-0.2, -0.7), 
                                  bbox_transform=coupling_ax.transAxes, borderpad=0.)
coupling_ax.add_artist(anchored_xbox)

#add legend
fig.legend(rl_legend_lines,rl_legend_text,loc=rl_legend_location,fancybox=True, framealpha=0.5,borderaxespad=0.,
                    ncol=1,handlelength=1, columnspacing=legend_column_spacing)

#add the panel labels
reward_ax.text(-0.105,0.86, r'\textbf{B}', transform=reward_ax.transAxes )
cvalue_ax.text(-0.105,0.84, r'\textbf{C}', transform=cvalue_ax.transAxes )
prot_0_ax.text(-0.385,0.8, r'\textbf{D}', transform=prot_0_ax.transAxes )
prot_final_ax.text(-0.13,0.86, r'\textbf{E}', transform=prot_final_ax.transAxes )
coupling_ax.text(-0.41,0.86, r'\textbf{F}', transform=coupling_ax.transAxes )

#save figure
plot_folder = os.path.join(log_dir,plotting.PLOT_DIR_NAME)
Path(plot_folder).mkdir(parents=True, exist_ok=True)
plt.savefig(plot_file_name, bbox_inches = 'tight')


#show
plt.show()

## Produce Fig. 4

In [ ]:
main_dir = "../paper_plot_data/qubit_refrigerator"

cycle_0_file = os.path.join(main_dir, "4/2022_03_11-11_50_21_aend=1.0/saved_policies/det_policy.txt")
cycle_1_file = os.path.join(main_dir, "1/2022_02_25-16_07_12_aend=0.8/saved_policies/det_policy.txt")
cycle_2_file = os.path.join(main_dir, "2/2022_02_26-23_03_54_aend=0.6/saved_policies/det_policy.txt")
cycle_3_file = os.path.join(main_dir, "3/2022_03_01-10_08_57_aend=0.4/saved_policies/det_policy.txt")
selected_cycle_cvals_index = [12, 8, 4, 0]
selected_cycle_repetition_index = [3,0, 1, 2]
actions_to_plot = 100 
actions_line_style = "scatter_plot"
marker_size=20
prot_linewidth = 12
extra_cycles_linewidth = 1.5
custom_colors=["black"]
inset_position = [0.687,0.675,0.12,0.12]
extra_cycles_0 = [lambda x,a=2,omega=0.05475,dt=0.982: 0.25*(1. + np.tanh(a*np.cos(omega*x*dt))/np.tanh(a)),
               [0,110],"black"]
extra_cycles_linewidth_0 = 0.8
extra_cycles_1 = [lambda x,a=2,omega=0.0512239,dt=0.982: 0.25*(1. + np.tanh(a*np.cos(omega*x*dt))/np.tanh(a)),
               [0,110],"black"]
extra_cycles_linewidth_1 = 0.8
extra_cycles_2 = [lambda x,a=2,omega=0.0472428,dt=0.982: 0.25*(1. + np.tanh(a*np.cos(omega*x*dt))/np.tanh(a)),
               [0,110],"black"]
extra_cycles_linewidth_2 = 0.6
rl_legend_lines=[Line2D([0], [0], color='black', linewidth=0, marker="o"), 
                    Line2D([0], [0], color='black', linewidth=1, dashes=(2/0.8,2/0.8))]
rl_legend_text=["RL","Trap."]
rl_legend_location=[0.48,0.328]
legend_column_spacing = 0.5
plot_file_name = "fig_4.pdf"

#font size
matplotlib.rcParams.update({'font.size': 14, "text.usetex": True,
                            'text.latex.preamble' : r'\usepackage{amsmath,amssymb,physics}'})

#setup the fig
fig = plt.figure(constrained_layout=True, figsize=(6,5.45))
gs0 = gridspec.GridSpec(2, 1, figure=fig)
gs1 = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=gs0[0], width_ratios=[1,0.4,1.6])
gs2 = gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=gs0[1],wspace=0.24)

#setup the axis
figmerit_ax = fig.add_subplot(gs1[0])
pareto_left_ax = fig.add_subplot(gs1[1])
pareto_right_ax = fig.add_subplot(gs1[2],sharey=pareto_left_ax)
prot_0_ax = fig.add_subplot(gs2[0,0])
prot_1_ax = fig.add_subplot(gs2[0,1],sharey=prot_0_ax)
prot_2_ax = fig.add_subplot(gs2[1,0],sharex=prot_0_ax)
prot_3_ax = fig.add_subplot(gs2[1,1],sharey=prot_2_ax, sharex=prot_1_ax)
plt.setp(prot_1_ax.get_yticklabels(), visible=False)
plt.setp(prot_3_ax.get_yticklabels(), visible=False)          
plt.setp(prot_0_ax.get_xticklabels(), visible=False)
plt.setp(prot_1_ax.get_xticklabels(), visible=False)          

#useful dicts to read data
quantity_ind = {'pow': 0, 'entropy': 1, 'eff':2, 'a': 3, 'h': 4}

#load data for plotting
otto_pareto_file = os.path.join(main_dir,"otto_pareto.txt")
rl_pareto_files = [os.path.join(main_dir, subdir, "det_pareto.txt") for subdir in os.listdir(main_dir) if
                          os.path.isdir(os.path.join(main_dir,subdir))]
otto_data = np.loadtxt(otto_pareto_file)
#sort each file by the value of the weight
rl_all_data = []
for i, rl_file in enumerate(rl_pareto_files):
    rl_data = np.loadtxt(rl_file)
    rl_all_data.append(rl_data[rl_data[:, quantity_ind['a']].argsort()])
#put everything in a big numpy array
rl_all_data = np.array(rl_all_data)
#create index of non-selected c-vals
non_selected_cycle_cvals_index = [i for i in range(rl_all_data.shape[1]) if not i in selected_cycle_cvals_index]  
    
#Figure of merit (panel a)
figmerit_ax.errorbar(rl_all_data[0,non_selected_cycle_cvals_index,quantity_ind["a"]],
                     np.mean(rl_all_data[:,non_selected_cycle_cvals_index,quantity_ind["h"]], axis=0),
                    yerr=0.5*np.std(rl_all_data[:,non_selected_cycle_cvals_index,quantity_ind["h"]], axis=0,ddof=1),
                     markersize=0, fmt="o", label="RL", c="black", capsize=3, linewidth=1, capthick=2)
figmerit_ax.errorbar(rl_all_data[0,selected_cycle_cvals_index,quantity_ind["a"]],
                     np.mean(rl_all_data[:,selected_cycle_cvals_index,quantity_ind["h"]], axis=0),
                    yerr=0.5*np.std(rl_all_data[:,selected_cycle_cvals_index,quantity_ind["h"]], axis=0,ddof=1),
                     markersize=0, fmt="o", c="cornflowerblue", capsize=3, linewidth=1, capthick=2)
figmerit_ax.scatter(otto_data[:,quantity_ind["a"]], otto_data[:,quantity_ind["h"]], s=marker_size,
                    marker="o",label="Trapezoidal",c="tomato") 
figmerit_ax.set_xlabel(r"$c$")
figmerit_ax.set_ylabel(r"$\ev{r_c}$")

#Pareto front (panel b)

#broken x-axis of the Pareto plot
pareto_left_ax.spines['right'].set_visible(False)
pareto_right_ax.spines['left'].set_visible(False)
pareto_right_ax.yaxis.tick_right()
plt.setp(pareto_right_ax.get_yticklabels(), visible=False)          

#add diagonal lines
d = .02 # how big to make the diagonal lines in axes coordinates
# arguments to pass plot, just so we don't keep repeating them
break_linewidth = 1
extra_tilt=3
kwargs = dict(transform=pareto_left_ax.transAxes, color='k', clip_on=False)
pareto_left_ax.plot((1-extra_tilt*d,1+extra_tilt*d), (-d,+d), linewidth=break_linewidth, **kwargs)
pareto_left_ax.plot((1-extra_tilt*d,1+extra_tilt*d),(1-d,1+d), linewidth=break_linewidth, **kwargs)
kwargs.update(transform=pareto_right_ax.transAxes)  # switch to the bottom axes
pareto_right_ax.plot((-d,+d), (1-d,1+d), linewidth=break_linewidth, **kwargs)
pareto_right_ax.plot((-d,+d), (-d,+d), linewidth=break_linewidth, **kwargs)

#plot quantities
pareto_right_ax.scatter(rl_all_data[:,:,quantity_ind["eff"]].reshape(-1),rl_all_data[:,:,quantity_ind["pow"]].reshape(-1),
                  s=marker_size, marker="o", label="RL", c="black")
pareto_right_ax.scatter(rl_all_data[selected_cycle_repetition_index,selected_cycle_cvals_index,quantity_ind["eff"]],
                  rl_all_data[selected_cycle_repetition_index,selected_cycle_cvals_index,quantity_ind["pow"]],
                  s=marker_size*2, marker="o", c="cornflowerblue")
pareto_right_ax.set_xlabel(r"$\eta_{\text{cool}}$")
pareto_left_ax.set_ylabel(r"$\ev*{P_\text{cool}}/P_0$")
pareto_left_ax.set_xlim([0., 0.1])
pareto_right_ax.set_xlim([0.65, 0.96])
pareto_right_ax.scatter(otto_data[:,quantity_ind["eff"]],otto_data[:,quantity_ind["pow"]], s=marker_size,
                  marker="o", label="Trapezoidal",c="tomato")
pareto_left_ax.scatter([0.2],[0.2], s=marker_size, marker="o", label="RL", c="black")
pareto_left_ax.scatter([0.2],[0.2], s=marker_size, marker="o", label="Trapezoidal",c="tomato")
pareto_left_ax.scatter([0.06], [12.9467/15000/(6.62*10**(-4))], marker="x", c="purple", s=marker_size*3,
                 label=r"Ref. [69]")
leg = pareto_left_ax.legend(framealpha=0.5, fancybox=True, handlelength=0.6, loc=(0.2,0.3))
leg.set_in_layout(False) #this is crucial such that constrained layout doesnt make space for the legend
pareto_left_ax.set_zorder(1)

#first protocol (panel c)
plotting.plot_actions_on_axis(cycle_0_file, prot_0_ax, actions_to_plot=actions_to_plot,custom_colors=custom_colors,
                                is_tri=False,k_notation=False, x_count_from_zero=True,
                                linewidth = prot_linewidth, xlabel="$t[dt]$",extra_cycles=extra_cycles_0,
                                extra_cycles_linewidth=extra_cycles_linewidth_0,hide_gray_vertical_line=False,
                                hide_xaxis_label=True,line_style=actions_line_style)

#second protocol (panel d)
plotting.plot_actions_on_axis(cycle_1_file, prot_1_ax, actions_to_plot=actions_to_plot,custom_colors=custom_colors,
                                is_tri=False, k_notation=False, x_count_from_zero=True,
                                linewidth = prot_linewidth, xlabel="$t[dt]$",extra_cycles=extra_cycles_1,
                                extra_cycles_linewidth=extra_cycles_linewidth_1,hide_gray_vertical_line=False,
                                hide_xaxis_label=True,hide_yaxis_label=True,line_style=actions_line_style)

#second protocol (panel e)
plotting.plot_actions_on_axis(cycle_2_file, prot_2_ax, actions_to_plot=actions_to_plot,custom_colors=custom_colors,
                                is_tri=False, k_notation=False, x_count_from_zero=True,
                                linewidth = prot_linewidth, xlabel="$t[dt]$",extra_cycles=extra_cycles_2,
                                extra_cycles_linewidth=extra_cycles_linewidth_2,hide_gray_vertical_line=False,
                                line_style=actions_line_style)

#second protocol (panel f)
plotting.plot_actions_on_axis(cycle_3_file, prot_3_ax, actions_to_plot=actions_to_plot,custom_colors=custom_colors,
                                is_tri=False, k_notation=False, x_count_from_zero=True,
                                linewidth = prot_linewidth, xlabel="$t[dt]$",hide_gray_vertical_line=False,
                                hide_yaxis_label=True,line_style=actions_line_style)

#legend
fig.legend(rl_legend_lines,rl_legend_text,loc=rl_legend_location,fancybox=False, framealpha=0.,borderaxespad=0.,
                   ncol=1,handlelength=0.6, columnspacing=legend_column_spacing)

#extra labels
figmerit_ax.text(0.04,0.05, r"\textbf{F}", transform=figmerit_ax.transAxes, color="cornflowerblue")
figmerit_ax.text(0.36,0.20, r"\textbf{E}", transform=figmerit_ax.transAxes, color="cornflowerblue")
figmerit_ax.text(0.69,0.47, r"\textbf{D}", transform=figmerit_ax.transAxes, color="cornflowerblue")
figmerit_ax.text(0.72,0.91, r"\textbf{C}", transform=figmerit_ax.transAxes, color="cornflowerblue")
pareto_right_ax.text(0.9,0.43, r"\textbf{F}", transform=pareto_right_ax.transAxes, color="cornflowerblue")
pareto_right_ax.text(0.62,0.6, r"\textbf{E}", transform=pareto_right_ax.transAxes, color="cornflowerblue")
pareto_right_ax.text(0.62,0.91, r"\textbf{D}", transform=pareto_right_ax.transAxes, color="cornflowerblue")
pareto_right_ax.text(0.24,0.91, r"\textbf{C}", transform=pareto_right_ax.transAxes, color="cornflowerblue")

# panel numbering labels
fig.text(-0.41, 0.93, r"\textbf{A}",transform=figmerit_ax.transAxes)
fig.text(-1., 0.93, r"\textbf{B}",transform=pareto_left_ax.transAxes)
fig.text(-0.23,0.85, r"\textbf{C}",transform=prot_0_ax.transAxes)
fig.text(-0.2,0.85, r"\textbf{D}",transform=prot_1_ax.transAxes)
fig.text(-0.23,0.85, r"\textbf{E}",transform=prot_2_ax.transAxes)
fig.text(-0.2,0.85, r"\textbf{F}",transform=prot_3_ax.transAxes)

#extra labelling
fig.text(0.8,0.81, r"$c=1$",transform=prot_0_ax.transAxes)
fig.text(0.74,0.81, r"$c=0.8$",transform=prot_1_ax.transAxes)
fig.text(0.74,0.81, r"$c=0.6$",transform=prot_2_ax.transAxes)
fig.text(0.74,0.81, r"$c=0.4$",transform=prot_3_ax.transAxes)

#save figure
plt.savefig(plot_file_name)
#show
plt.show()

## Produce Fig. 8

In [ ]:
main_dir = "../paper_plot_data/qubit_refrigerator"

cycle_0_file = os.path.join(main_dir, "5/2022_03_13-23_12_52_aend=1.0/saved_policies/det_policy.txt")
cycle_1_file = os.path.join(main_dir, "5/2022_03_12-14_58_20_aend=0.8/saved_policies/det_policy.txt")
cycle_2_file = os.path.join(main_dir, "5/2022_03_12-14_56_54_aend=0.6/saved_policies/det_policy.txt")
cycle_3_file = os.path.join(main_dir, "5/2022_03_21-10_30_55_aend=0.4/saved_policies/det_policy.txt")
actions_to_plot = 100 
actions_line_style = "scatter_plot"
prot_linewidth = 12
extra_cycles_linewidth = 1.5
custom_colors=["black"]
extra_cycles_0 = [lambda x,a=2,omega=0.05475,dt=0.982: 0.25*(1. + np.tanh(a*np.cos(omega*x*dt))/np.tanh(a)),
               [0,110],"black"]
extra_cycles_linewidth_0 = 0.8
extra_cycles_1 = [lambda x,a=2,omega=0.0512239,dt=0.982: 0.25*(1. + np.tanh(a*np.cos(omega*x*dt))/np.tanh(a)),
               [0,110],"black"]
extra_cycles_linewidth_1 = 0.8
extra_cycles_2 = [lambda x,a=2,omega=0.0472428,dt=0.982: 0.25*(1. + np.tanh(a*np.cos(omega*x*dt))/np.tanh(a)),
               [0,110],"black"]
extra_cycles_linewidth_2 = 0.6
rl_legend_lines=[Line2D([0], [0], color='black', linewidth=0, marker="o"), 
                    Line2D([0], [0], color='black', linewidth=1, dashes=(2/0.8,2/0.8))]
rl_legend_text=["RL","Trap."]
rl_legend_location=[0.0,0.98]
legend_column_spacing = 0.5
plot_file_name = "fig_8.pdf"

#font size
matplotlib.rcParams.update({'font.size': 14, "text.usetex": True,
                            'text.latex.preamble' : r'\usepackage{amsmath,amssymb,physics}'})

#create the figure and subplots
fig = plt.figure(constrained_layout=True, figsize=(6,3.3))
gs = gridspec.GridSpec(2, 2, figure=fig, height_ratios = [0.9,0.9])
prot_0_ax = fig.add_subplot(gs[0, 0])
prot_1_ax = fig.add_subplot(gs[0, 1],sharey=prot_0_ax)
prot_2_ax = fig.add_subplot(gs[1, 0],sharex=prot_0_ax)
prot_3_ax = fig.add_subplot(gs[1, 1],sharey=prot_2_ax, sharex=prot_1_ax)
plt.setp(prot_1_ax.get_yticklabels(), visible=False)
plt.setp(prot_3_ax.get_yticklabels(), visible=False)          
plt.setp(prot_0_ax.get_xticklabels(), visible=False)
plt.setp(prot_1_ax.get_xticklabels(), visible=False)          

#useful dicts to read data
quantity_ind = {'pow': 0, 'entropy': 1, 'eff':2, 'a': 3, 'h': 4}

#first protocol (panel a)
plotting.plot_actions_on_axis(cycle_0_file, prot_0_ax, actions_to_plot=actions_to_plot,custom_colors=custom_colors,
                                is_tri=False,k_notation=False, x_count_from_zero=True,
                                linewidth = prot_linewidth, xlabel="$t[dt]$",extra_cycles=extra_cycles_0,
                                extra_cycles_linewidth=extra_cycles_linewidth_0,hide_gray_vertical_line=False,
                                hide_xaxis_label=True,line_style=actions_line_style)

#second protocol (panel b)
plotting.plot_actions_on_axis(cycle_1_file, prot_1_ax, actions_to_plot=actions_to_plot,custom_colors=custom_colors,
                                is_tri=False, k_notation=False, x_count_from_zero=True,
                                linewidth = prot_linewidth, xlabel="$t[dt]$",extra_cycles=extra_cycles_1,
                                extra_cycles_linewidth=extra_cycles_linewidth_1,hide_gray_vertical_line=False,
                                hide_xaxis_label=True,hide_yaxis_label=False,line_style=actions_line_style)

#second protocol (panel c)
plotting.plot_actions_on_axis(cycle_2_file, prot_2_ax, actions_to_plot=actions_to_plot,custom_colors=custom_colors,
                                is_tri=False, k_notation=False, x_count_from_zero=True,
                                linewidth = prot_linewidth, xlabel="$t[dt]$",extra_cycles=extra_cycles_2,
                                extra_cycles_linewidth=extra_cycles_linewidth_2,hide_gray_vertical_line=False,
                                line_style=actions_line_style)

#second protocol (panel d)
plotting.plot_actions_on_axis(cycle_3_file, prot_3_ax, actions_to_plot=actions_to_plot,custom_colors=custom_colors,
                                is_tri=False, k_notation=False, x_count_from_zero=True,
                                linewidth = prot_linewidth, xlabel="$t[dt]$",hide_gray_vertical_line=False,
                                hide_yaxis_label=False,line_style=actions_line_style)

#legend
prot_0_ax.legend(rl_legend_lines,rl_legend_text,loc=rl_legend_location,fancybox=False, framealpha=0.,borderaxespad=0.,
                    ncol=2,handlelength=0.6, columnspacing=legend_column_spacing)


# panel numbering labels
fig.text(-0.22,0.87, r"\textbf{A}",transform=prot_0_ax.transAxes)
fig.text(-0.1,0.87, r"\textbf{B}",transform=prot_1_ax.transAxes)
fig.text(-0.22,0.87, r"\textbf{C}",transform=prot_2_ax.transAxes)
fig.text(-0.1,0.87, r"\textbf{D}",transform=prot_3_ax.transAxes)

#extra labelling
fig.text(0.81,0.85, r"$c=1$",transform=prot_0_ax.transAxes)
fig.text(0.75,0.85, r"$c=0.8$",transform=prot_1_ax.transAxes)
fig.text(0.75,0.846, r"$c=0.6$",transform=prot_2_ax.transAxes)
fig.text(0.75,0.846, r"$c=0.4$",transform=prot_3_ax.transAxes)

#save figure
plt.savefig(plot_file_name)
#show
plt.show()